In [1]:
import numpy as np
import wisps
import wisps.simulations as wispsim
import pandas as pd
import splat
import matplotlib.pyplot as plt
import astropy.units as u
from astropy.coordinates import SkyCoord
from scipy.interpolate import interp1d
import numba
from tqdm import tqdm
import splat.empirical as spem
import wisps.simulations.effective_numbers as ef 
import seaborn as sns
from matplotlib.ticker import MultipleLocator
from matplotlib.colors import Normalize
import matplotlib as mpl
import splat.empirical as spe

%matplotlib inline

In [2]:
pnts=wisps.OBSERVED_POINTINGS

In [3]:
def bin_by_spt_bin(sp_types, number):
    ranges=[[17, 20], [20, 25], [25, 30], [30, 35], [35, 40]]
    numbers=[]
    for r in ranges:
        idx= np.logical_and((r[0]<=sp_types), (r[1]>sp_types))
        numbers.append(np.nansum(number[idx]))
    return numbers

def iswithin_mag_limits(mags, pnt):
    #mgs is a dictionary
    flags=[]
    for k in pnt.mag_limits.keys():
        if k =='F110' and pnt.survey =='hst3d':
            flags.append(True)
        else:
            flags.append(mags[k] < pnt.mag_limits[k])
    return np.logical_or.reduce(flags)


In [4]:
sgrid=wispsim.SPGRID

In [5]:
cands=pd.read_hdf(wisps.OUTPUT_FILES+'/true_spectra_cands.hdf', key='with_indices')

In [6]:
pnt_names=[x.name for x in pnts]

In [7]:
pnt_names.index('par1')

149

In [8]:
pnts[0].survey

'hst3d'

In [9]:
def get_pointing(grism_id):
    if grism_id.startswith('par'):
        pntname=grism_id.lower().split('-')[0]
    else:
        pntname=grism_id.lower().split('-g141')[0]
    loc=pnt_names.index(pntname)
    return np.array(pnts)[loc]
    

def get_mags(row):
    return pd.Series(row.spectra.mags)

In [10]:
mags_df=cands.apply(get_mags, axis=1)
tab=wisps.Annotator.reformat_table(cands.join(mags_df))
tab['pnt']=tab['grism_id'].apply(get_pointing)

In [11]:
obsmgs=tab[['F140W', 'F110W', 'F160W']].rename(columns={"F110W": "F110", 
                                                                    "F140W": "F140",
                                                                    "F160W": "F160"}).to_dict('records')
flags=[iswithin_mag_limits(x, y) for x, y in zip(obsmgs, tab.pnt.values)]

cdf_to_use=tab[flags]
nobs=wisps.custom_histogram(cdf_to_use.spt.apply(wisps.make_spt_number), sgrid, 1)

In [12]:
spgrid2=['M7-L0', 'L0-L5', 'L5-T0', 'T0-T5', 'T5-Y0']
cnorm=Normalize(wispsim.HS[0], wispsim.HS[-1])

In [13]:
#data=ef.simulation_outputs()['baraffe03']
#data['mass']

In [14]:
def compute_simulated_numbers(hidx, model='saumon', selection='prob'):
    #an index in accordance with the scale height
    data=ef.simulation_outputs()[model]

    
    simdf=pd.DataFrame()
    simdf['spt']=data['spts'].flatten()
    simdf['mass']=data['mass'].flatten()
    simdf['slprob']=data['sl_prob'][hidx]
    simdf['snr']=data['snrjs'][hidx]
    simdf['appF140']=data['appf140s'][hidx]
    simdf['appF110']=data['appf110s'][hidx]
    simdf['appF160']=data['appf160s'][hidx]
    simdf['pntname']=data['pointing'][hidx]
    
    simdf['pnt']=simdf.pntname.apply(lambda x: np.array(pnts)[pnt_names.index(x)])
    
    simmgs=simdf[['appF140', 'appF110', 'appF160']].rename(columns={"appF110": "F110", 
                                                                    "appF140": "F140",
                                                                    "appF160": "F160"}).to_dict('records')
    flags=[iswithin_mag_limits(x, y) for x, y in zip(simmgs,  simdf.pnt.values)]
    
    
    cutdf=(simdf[flags]).reset_index(drop=True)
    NORM = 2.0*(10**-3)/ len(cutdf.mass[np.logical_and(cutdf.mass>=.09, cutdf.mass <=.1)])
    
    NSIM=dict(zip(wispsim.SPGRID,np.zeros(len(wispsim.SPGRID))))
    #rounded spectral type
    cutdf['spt_r']=cutdf.spt.apply(np.round)
    for g in cutdf.groupby('spt_r'):
        NSIM[g[0]]=np.nansum((g[1]).slprob*NORM)
        
    return NSIM

In [15]:
volumes=[]
for pnt in pnts:
    vs=[]
    for h in wispsim.HS:
        vsx=[]
        for g in wispsim.SPGRID:
            vsx.append((pnt.volumes[h])[g])
        vs.append(vsx)
    volumes.append(vs)
volumes=np.array(volumes)

In [16]:
VOLUMES=np.nansum(volumes, axis=0)*4.1*(u.arcmin**2).to(u.radian**2)

In [17]:
import seaborn as sns
flatui =['#001f3f', '#0074D9', '#7FDBFF', '#39CCCC', '#3D9970', '#2ECC40', '#01FF70', '#FFDC00']
sns.set_palette(sns.color_palette(flatui, len(wispsim.HS)))

In [ ]:
fig, (ax, ax1, ax2)=plt.subplots(figsize=(14, 8), ncols=3, sharey=True, sharex=True)


nexpct=[]
for idx, h in tqdm(enumerate(wispsim.HS)):
    
    nsimulated_saumon=compute_simulated_numbers(idx, model='saumon2008')
    nsimulated_baraffe=compute_simulated_numbers(idx, model='baraffe2003' )
    nsimulated_sonora=compute_simulated_numbers(idx, model='marley2019' )
    
    ns_bar=[]
    ns_saumon=[]
    ns_sonora=[]
    for s in wispsim.SPGRID:
            ns_bar.append(nsimulated_baraffe[s])
            ns_saumon.append(nsimulated_saumon[s])
            ns_sonora.append(nsimulated_sonora[s])
    
        
    ns_bar=np.array(ns_bar)
    ns_saumon=np.array(ns_saumon)
    ns_sonora=np.array(ns_sonora)
    
    ax1.step(spgrid2, bin_by_spt_bin(wispsim.SPGRID,ns_bar*VOLUMES[idx]), 
                 where='mid', label='h={} pc'.format(wispsim.HS[idx]))
    
    ax.step(spgrid2, bin_by_spt_bin(wispsim.SPGRID,ns_saumon*VOLUMES[idx]),
            where='mid', label='h={} pc'.format(wispsim.HS[idx]))
    
    ax2.step(spgrid2, bin_by_spt_bin(wispsim.SPGRID,ns_sonora*VOLUMES[idx]),
            where='mid', label='h={} pc'.format(wispsim.HS[idx]))
    
    nexpct.append( bin_by_spt_bin(wispsim.SPGRID,ns_bar*VOLUMES[idx]))
    



ax.set_yscale('log')
ax1.set_yscale('log')
ax2.set_yscale('log')

ax.set_ylim([0.6, 1000])
ax1.set_ylim([0.6, 1000])
ax2.set_ylim([0.6, 1000])

y2=bin_by_spt_bin(wispsim.SPGRID,nobs)
dy2=np.sqrt(y2)
ax1.errorbar(spgrid2,y2, yerr=dy2,
            label='observations',fmt='o', color='k')
ax.errorbar(spgrid2,y2, yerr=dy2,
            label='observations',fmt='o', color='k')
ax2.errorbar(spgrid2,y2, yerr=dy2,
            label='observations',fmt='o', color='k')




ax1.set_xlabel('SpT',fontsize=18)

ax.set_xlabel('SpT',fontsize=18)
ax.set_ylabel('N',fontsize=18)

ax.set_title('Model= S03', fontsize=18)
ax1.set_title('Model= B03', fontsize=18)
ax2.set_title('Model= Sonora', fontsize=18)
ax.legend(fontsize=14, loc='lower left')

plt.minorticks_on()

plt.tight_layout()
plt.savefig(wisps.OUTPUT_FIGURES+'/oberved_numbers.pdf', bbox_inches='tight')


3it [17:13, 343.62s/it]